In [133]:
# Author: Joshua Gregory
# Date: Sept. 2025
# Descr: Uses BFS & DFS (Iteratively Deepening) to solve 8-Puzzle Problem variant.

In [53]:
from collections import deque
import random

In [54]:
GOAL = [[1,2,3], [8,0,4], [7,6,5]]  # solution state as 2d arr (0 = blank)
N = 3                               # size of state space = NxN

row_moves = [0, 0, -1, 1]    # L/R/U/D values for row movements
col_moves = [-1, 1, 0, 0]    # L/R/U/D values for col movements

In [55]:
# Puzzle state space, as 2d arr = [[0,1,2], [3,4,5], [6,7,8]]
class EightPuzzle:
  def __init__(self, state, x, y, level):
    self.state = state    # state space represented as 2d array
    self.x = x            # x (row) coord of blank tile (zero)
    self.y = y            # y (col) coord of blank tile (zero)
    self.level = level    # depth level (ie: # moves from root)

In [56]:
# Randomize grid with values [0,8], return grid & xy coords of blank tile (0)
def randomize_puzzle_state():
  tile_values = list(range(9))  # list of integers [0,8], 0 = blank tile
  random.shuffle(tile_values)   # shuffle tile positions

  puzzle_state = [tile_values[i : i+3] for i in range(0, 9, 3)]  # convert to grid

  for x in range(3):
    for y in range(3):               # search each col in earch row
      if puzzle_state[x][y] == 0:    # for blank tile (value = 0)
        return puzzle_state, x, y    # return grid & xy coords of blank tile

In [57]:
# Check if current state matches solution state
def is_goal(state):
  return state == GOAL

In [58]:
# Validate move is legal
def is_legal_move(x, y):
  return 0 <= x < N and 0 <= y < N

In [112]:
# Print 2d array state space as an NxN grid with space for blank tile (0)
def to_grid(state):
  print("--------")
  for row in state:
    print(' '.join(' ' if val == 0 else str(val) for val in row))
  print("--------")

In [123]:
# Swap blank tile using row_moves/col_moves
def update_state(state, curr_x, curr_y, move_x, move_y):
  new_state = [row[:] for row in state]  # copy state by row then swap coords of blank (0) tile & neighbor
  new_state[curr_x][curr_y], new_state[move_x][move_y] = new_state[move_x][move_y], new_state[curr_x][curr_y]
  return new_state


In [130]:
# DFS
def dfs_solver(state, x, y):
  STACK = []     # LIFO stack for exploring branch depths iteratively
  SEEN = set()   # re-init per instance, stores seen states to prevent cycling

  STACK.append(EightPuzzle(state, x, y, 0))  # push root to stack
  SEEN.add(tuple(map(tuple, state)))         # add to set of seen nodes

  while STACK:              # while nodes (states) remain on stack
    current = STACK.pop()   # pop the most recently added node (state)

    print(f"Depth = {current.level}:")  # print current depth
    to_grid(current.state)               # print state as NxN grid

    if is_goal(current.state):
      print(f"Solution found at depth {current.level}.")  # solution state reached
      return  # exit

    for i in range(4):                   # move blank tile
      move_x = current.x + row_moves[i]  # new x = current + L/R/U/D
      move_y = current.y + col_moves[i]  # new y = current + L/R/U/D

      if is_legal_move(move_x, move_y):  # validate move & update state
        new_state = update_state(current.state, current.x, current.y, move_x, move_y)

        state_tuple = tuple(map(tuple, new_state))  # convert to tuples

        if state_tuple not in SEEN:
          SEEN.add(state_tuple)   # add visited state to set, push state to stack
          STACK.append(EightPuzzle(new_state, move_x, move_y, current.level+1))

  print("No solution found.")  # all possible moves exhausted

In [129]:
def bfs_solver(state, x, y):
  QUE = deque()  # FIFO queue for searching complete tiers
  SEEN = set()   # re-init per instance, prevents cycling

  QUE.append(EightPuzzle(state, x, y, 0))  # init root & enqueue
  SEEN.add(tuple(map(tuple, state)))       # add tuple to visited nodes

  while QUE:                  # while states remain in queue
    current = QUE.popleft()   # pop firstmost node

    print(f"Depth = {current.level}:")  # print current depth
    to_grid(current.state)             # print current state as NxN grid

    if is_goal(current.state):
      print(f"Solution found at depth {current.level}")  # solution state reached
      return  # exit

    for i in range(4):                   # move blank tile
      move_x = current.x + row_moves[i]  # new x = current + L/R/U/D
      move_y = current.y + col_moves[i]  # new y = current + L/R/U/D

      if is_legal_move(move_x, move_y):  # validate move & update state
        new_state = update_state(current.state, current.x, current.y, move_x, move_y)

        state_tuple = tuple(map(tuple, new_state))     # updated state space

        if state_tuple not in SEEN:
          SEEN.add(state_tuple)   # add visited state to set, push state to stack
          QUE.append(EightPuzzle(new_state, move_x, move_y, current.level+1))

  print("No solution found.")  # all possible moves exhausted

In [126]:
# Helper method to display menu & return validated user input
def get_menu_choice():
  while True:   # loop menu & prompt until valid input
    print("\n[8-PUZZLE BFS/DFS SOLVER MAIN MENU]")
    print("-----------------------------------")
    print("1. Generate a random 8-puzzle board")
    print("2. Solve via BFS (Breadth First Search)")
    print("3. Solve via DFS (Depth First Search with Iterative Deepening)")
    print("4. Exit program")
    choice = input("Enter a menu option: ").strip()  # save input without spaces

    if choice == '1' or choice == '2' or choice == '3' or choice == '4':
      return choice
    print("\nInvalid input. Enter a valid menu number: 1, 2, 3, or 4.")


In [127]:
# Driver method with console ui
def main():
  puzzle = None      # default null (prevent BFS or DFS)
  x, y = None, None  # default null

  while True:
    choice = get_menu_choice()  # display menu prompt & return valid input

    # Menu 1: Generate new 8-Puzzle Board
    if choice == '1':
      puzzle, x, y = randomize_puzzle_state()  # create randomized board
      print("\nNew 8-Puzzle board created:")
      to_grid(puzzle)  # print initial puzzle state

    # Menu 2: Solve via BFS
    if choice == '2':
      if not puzzle:
        print("\nRequires an 8-Puzzle board! Use menu option 1 first.")
      else:
        print()  # formatting
        bfs_solver(puzzle, x, y)  # call bfs, pass state & blank tile coords

    # Menu 3: Solve via DFS
    if choice == '3':
      if not puzzle:
        print("\nRequires an 8-Puzzle board! Use menu option 1 first.")
      else:
        print()  # formatting
        dfs_solver(puzzle, x, y)  # call dfs, pass state & blank tile coords

    # Menu 4: Exit
    if choice == '4':
      print("\nExiting program... Goodbye!")

In [ ]:
# Tester
if __name__ == '__main__':
  main()